# Time-Series Forecasting – Bitcoin (BTC) Price Prediction

## 1. Business Understanding



### 1.1 Context

Bitcoin adalah aset digital yang dikenal memiliki pergerakan harga yang sangat naik-turun. Dalam waktu singkat, harganya bisa naik atau turun cukup drastis. Hal ini berbeda dengan aset tradisional seperti emas yang cenderung lebih stabil.

Perubahan harga Bitcoin dipengaruhi oleh banyak hal, seperti sentimen pasar, perkembangan teknologi blockchain, aturan pemerintah di berbagai negara, kondisi ekonomi global, hingga perilaku investor yang bersifat spekulatif. Karena sifatnya yang tidak stabil inilah, Bitcoin menjadi objek yang menarik untuk dipelajari, terutama dalam konteks prediksi harga menggunakan data historis.

Prediksi harga Bitcoin memiliki manfaat penting, misalnya untuk membantu trader dan investor dalam mengambil keputusan, mengelola risiko portofolio, serta memahami tren jangka menengah hingga panjang.

### 1.2 Problem Statement

Stakeholder ingin mengetahui apakah data historis harga Bitcoin dapat digunakan untuk memprediksi harga Bitcoin di masa depan, khususnya harga penutupan (*closing price*) dalam skala bulanan. Data yang digunakan adalah data harga harian Bitcoin dari tahun 2014 hingga 2025.

Beberapa pertanyaan utama yang ingin dijawab melalui proyek ini antara lain:
- Apakah data harga Bitcoin di masa lalu cukup berguna untuk memprediksi harga di masa depan?
- Model prediksi mana yang bekerja lebih baik untuk data dengan pergerakan harga yang sangat tidak stabil?
- Seberapa besar kesalahan prediksi yang dihasilkan oleh masing-masing model?

### 1.3 Goals

Tujuan dari proyek ini adalah membangun proses prediksi harga Bitcoin secara menyeluruh menggunakan pendekatan *time-series forecasting*. Secara lebih spesifik, tujuan yang ingin dicapai adalah:
- Mengubah data harga Bitcoin harian menjadi data bulanan agar pola pergerakan harga lebih mudah diamati.
- Memprediksi harga Bitcoin bulanan untuk dua tahun ke depan (24 bulan).
- Membandingkan hasil prediksi dari beberapa model *time-series* yang berbeda.
- Menilai tingkat akurasi setiap model untuk mengetahui model mana yang paling baik digunakan.


In [ ]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root added to sys.path:", PROJECT_ROOT)

## 2. Data Acquisition and Understanding

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

### 2.1 Load Data

In [ ]:
# Load dataset
try:
    df_raw = pd.read_csv('../datasets/btc_2014_2025.csv')
    df = df_raw.copy()
    # Convert 'date' to datetime
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    print("Original Data Shape (Daily):", df.shape)
    display(df.head())
except FileNotFoundError:
    print("Error: File 'btc_2014_2025.csv' not found. Please upload the dataset.")

Menggunakan dataset `btc_2014_2025.csv` yang berisi data harian **OHLCV** Bitcoin yang diambil dari Yahoo Finance. Dataset ini mencatat pergerakan harga Bitcoin setiap hari, mulai dari harga pembukaan hingga penutupan, serta jumlah transaksi yang terjadi.

Secara sederhana, data OHLCV terdiri dari:
- **Open**: harga Bitcoin saat perdagangan hari tersebut dimulai
- **High**: harga tertinggi yang tercapai dalam satu hari
- **Low**: harga terendah dalam satu hari
- **Close**: harga Bitcoin di akhir hari perdagangan
- **Volume**: jumlah transaksi Bitcoin pada hari tersebut

Data yang digunakan mencakup periode panjang, yaitu dari tahun **2014 hingga 2025**. Rentang waktu ini cukup untuk melihat bagaimana harga Bitcoin bergerak dalam jangka panjang, termasuk periode kenaikan tajam, penurunan signifikan, serta masa volatilitas tinggi dan rendah.

Dalam proyek ini, data harga harian tidak langsung digunakan untuk melakukan prediksi. Data terlebih dahulu diolah menjadi data **bulanan** agar fluktuasi harian yang sangat ekstrem dapat dikurangi. Dengan cara ini, pola pergerakan harga Bitcoin menjadi lebih mudah diamati dan dianalisis.

Fokus utama analisis berada pada kolom **harga penutupan (close)**. Harga penutupan dipilih karena dianggap paling mewakili nilai Bitcoin pada akhir periode perdagangan dan umum digunakan sebagai acuan dalam analisis pasar dan pemodelan *time-series*.

Setelah struktur dan isi dataset dipahami, tahap selanjutnya adalah melakukan eksplorasi data untuk melihat tren umum, pola musiman, serta karakteristik dasar pergerakan harga Bitcoin sebelum masuk ke tahap pemodelan.

In [ ]:
df.info()

Output `df_raw.info()` digunakan untuk melihat gambaran umum struktur dataset yang digunakan dalam proyek ini.

Dataset terdiri dari **4.115 baris data**, yang merepresentasikan data harga Bitcoin **harian** dari tanggal **17 September 2014 hingga 22 Desember 2025**. Setiap baris mewakili satu hari perdagangan Bitcoin.

Dataset ini memiliki **5 kolom utama**, yaitu:
- **open**: harga Bitcoin saat hari perdagangan dimulai
- **high**: harga tertinggi Bitcoin dalam satu hari
- **low**: harga terendah Bitcoin dalam satu hari
- **close**: harga Bitcoin saat hari perdagangan berakhir
- **volume**: jumlah transaksi Bitcoin pada hari tersebut

Seluruh kolom pada dataset memiliki nilai yang **lengkap (non-null)**. Artinya, tidak ada data yang hilang pada periode waktu yang dianalisis. Hal ini penting karena data yang lengkap mengurangi kebutuhan proses pembersihan data (*data cleaning*) dan meminimalkan potensi bias pada hasil analisis.

In [ ]:
df.duplicated().sum()

Pengecekan data duplikat dilakukan untuk memastikan tidak ada data yang tercatat lebih dari satu kali pada periode yang sama. Hasil pengecekan menunjukkan nilai **0**, yang berarti **tidak ditemukan data duplikat** pada dataset. Setiap periode waktu hanya memiliki satu catatan harga Bitcoin yang unik.

### 2.2 Preprocessing: Resampling to Monthly

In [ ]:
# Resample to Monthly Mean price
df = df['close'].resample('MS').mean().to_frame(name='Price')

print("Resampled Data Shape (Monthly):", df.shape)
df.head()

Karena model ARIMA/SARIMA akan sangat lambat dan *noisy* jika menggunakan ribuan data harian, kita akan melakukan *resampling* menjadi data Bulanan (Monthly Start - MS). Kita mengambil rata-rata harga `close` untuk setiap bulan. Pada langkah ini, data harga Bitcoin yang semula bersifat **harian** diubah menjadi data **bulanan**. Proses ini disebut *resampling*, yaitu mengelompokkan data berdasarkan periode waktu tertentu.

Dalam kasus ini, yang digunakan adalah:
- **Harga penutupan (close)** sebagai fokus utama
- Periode **bulanan**, dengan mengambil **rata-rata harga penutupan setiap bulan**

Tujuan utama dari langkah ini adalah untuk mengurangi fluktuasi harian yang sangat tajam. Pergerakan harga Bitcoin dari hari ke hari sering kali terlalu ekstrem dan “berisik”, sehingga sulit untuk melihat pola pergerakan harga secara umum.

Dengan mengubah data menjadi bulanan:
- Tren jangka menengah dan panjang menjadi lebih jelas
- Pola naik dan turun harga lebih mudah diamati
- Model prediksi dapat bekerja lebih stabil dan tidak terlalu dipengaruhi lonjakan harian yang ekstrem

Hasil dari proses ini adalah dataset baru yang berisi satu nilai harga untuk setiap bulan, yang diberi nama **Price**. Setiap baris data sekarang mewakili rata-rata harga Bitcoin dalam satu bulan.

Dari output yang ditampilkan, terlihat bahwa data bulanan dimulai dari **September 2014** dan berlanjut secara berurutan setiap awal bulan. Dataset ini akan digunakan sebagai dasar utama untuk analisis lanjutan dan pemodelan *time-series* pada tahap berikutnya.


### 2.3 Exploratory Data Analysis (EDA)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['Price'],
        mode='lines',
        name='BTC Monthly Average Price',
        line=dict(color='orange', width=2)
    )
)

fig.update_layout(
    title='Bitcoin Monthly Price Trend (2014 - 2025)',
    xaxis_title='Year',
    yaxis_title='Price (USD)',
    template='plotly_white',
    hovermode='x unified',
    xaxis=dict(
        showgrid=True,
        rangeslider=dict(visible=True)
    ),
    yaxis=dict(showgrid=True)
)

fig.show()

Pada tahap ini, dilakukan visualisasi untuk melihat **tren harga Bitcoin bulanan** dari tahun 2014 hingga 2025. Grafik yang ditampilkan merupakan grafik garis (*line chart*) yang menunjukkan rata-rata harga Bitcoin setiap bulan. Sumbu horizontal (X) merepresentasikan waktu (tahun), sedangkan sumbu vertikal (Y) menunjukkan harga Bitcoin dalam satuan USD.

Tujuan utama dari visualisasi ini adalah untuk:
- Melihat arah pergerakan harga Bitcoin secara umum dalam jangka panjang
- Mengidentifikasi periode kenaikan dan penurunan harga yang signifikan
- Memahami tingkat volatilitas harga Bitcoin dari waktu ke waktu

Dari grafik tersebut, dapat diamati bahwa harga Bitcoin mengalami fluktuasi yang cukup tajam, dengan beberapa periode kenaikan yang sangat cepat diikuti oleh penurunan yang signifikan. Pola ini menunjukkan bahwa Bitcoin memiliki karakteristik volatilitas tinggi, sehingga menantang untuk diprediksi.

### 2.4 Stationarity Check

In [ ]:
def adf_test(series):
    result = adfuller(series.dropna())
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    if result[1] > 0.05:
        print("Result: Data is Non-Stationary (Series has a unit root)")
    else:
        print("Result: Data is Stationary")

adf_test(df['Price'])

Pengecekan stasioneritas dilakukan menggunakan **Augmented Dickey-Fuller (ADF) Test**. Uji ini bertujuan untuk mengetahui apakah data harga Bitcoin bersifat **stasioner** atau tidak.

Secara sederhana, data disebut **stasioner** jika pola pergerakannya relatif stabil dari waktu ke waktu, baik dari sisi rata-rata maupun tingkat variasinya. Sebaliknya, data disebut **tidak stasioner** jika memiliki tren naik atau turun yang kuat, sehingga pola statistiknya berubah seiring waktu.

Hasil ADF Test pada data harga Bitcoin bulanan menunjukkan:
- **ADF Statistic** bernilai negatif kecil
- **p-value sebesar 0.82**, yang jauh lebih besar dari batas umum 0.05

Berdasarkan hasil tersebut, data dinyatakan **tidak stasioner**. Artinya, harga Bitcoin memiliki tren jangka panjang dan perubahan pola yang signifikan dari waktu ke waktu.

Pengecekan stasioneritas ini sangat penting, terutama untuk model **ARIMA**, karena:
- ARIMA mengasumsikan data yang digunakan bersifat stasioner
- Data yang tidak stasioner dapat menyebabkan hasil prediksi yang menyesatkan
- Model menjadi sulit menangkap pola yang konsisten

Karena data harga Bitcoin belum stasioner, maka diperlukan proses **differencing** (menghitung selisih antar periode waktu) sebelum membangun model ARIMA. Proses ini bertujuan untuk menghilangkan tren dan membuat data lebih stabil. Dengan memahami karakteristik ini sejak awal, pemodelan dapat dilakukan dengan pendekatan yang tepat dan hasil prediksi menjadi lebih dapat diandalkan.

## 3. Modeling

### 3.1 Data Preparation (Train-Test Split)

In [ ]:
test_size = 24
train_data = df.iloc[:-test_size]
test_data = df.iloc[-test_size:]

print(f"Train period: {train_data.index.min()} to {train_data.index.max()}")
print(f"Test period: {test_data.index.min()} to {test_data.index.max()}")

Pada tahap ini, data dibagi menjadi dua bagian, yaitu **data latih (training set)** dan **data uji (test set)**. Pembagian ini bertujuan untuk menguji kemampuan model dalam memprediksi data yang belum pernah dilihat sebelumnya. Sebanyak **24 bulan terakhir** (setara dengan **2 tahun**) digunakan sebagai data uji. Sisanya digunakan sebagai data latih. Pendekatan ini umum digunakan dalam analisis *time-series* karena mencerminkan kondisi nyata, di mana model dilatih menggunakan data masa lalu dan diuji untuk memprediksi periode waktu ke depan.

Berdasarkan hasil pembagian data:
- **Data latih** mencakup periode dari **September 2014 hingga Desember 2023**
- **Data uji** mencakup periode dari **Januari 2024 hingga Desember 2025**

Data latih digunakan untuk mempelajari pola historis harga Bitcoin, seperti tren jangka panjang dan fluktuasi umum. Sementara itu, data uji digunakan untuk mengevaluasi seberapa baik model mampu memprediksi pergerakan harga Bitcoin pada periode yang lebih baru.


### 3.2 Model 1: ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit ARIMA Model
# Menggunakan order umum (1,1,1) untuk demonstrasi. 
# Disarankan menggunakan pmdarima.auto_arima untuk order optimal.
arima_model = ARIMA(train_data['Price'], order=(1, 1, 1))
arima_fit = arima_model.fit()
print(arima_fit.summary())

Pada tahap ini, dibangun model **ARIMA** sebagai model awal (*baseline*) untuk memprediksi harga Bitcoin bulanan. Model *baseline* digunakan sebagai pembanding agar performa model lain yang lebih kompleks dapat dievaluasi secara adil. Konfigurasi yang digunakan adalah **ARIMA(1,1,1)** karena cukup umum dipakai sebagai titik awal dalam analisis *time-series*.

Secara konsep, ARIMA(1,1,1) bekerja dengan tiga komponen utama:
- **Autoregressive (AR)**: model melihat hubungan antara harga bulan ini dengan harga pada satu bulan sebelumnya
- **Integrated (I)**: data harga diubah menjadi **selisih antar bulan (differencing)** untuk menghilangkan tren jangka panjang dan membuat pola data lebih stabil
- **Moving Average (MA)**: model memperhitungkan kesalahan prediksi pada periode sebelumnya sebagai bagian dari proses prediksi

Pendekatan ini dipilih karena harga Bitcoin memiliki karakteristik volatilitas yang tinggi dan perubahan tren yang cukup tajam. Dengan melakukan *differencing* secara implisit melalui parameter `d = 1`, model dapat fokus mempelajari pola perubahan harga, bukan nilai harga mentah yang terus meningkat atau menurun seiring waktu.

Hasil ringkasan model menunjukkan bahwa komponen **Moving Average (MA)** memiliki pengaruh yang lebih kuat dibandingkan komponen **Autoregressive (AR)**. Hal ini mengindikasikan bahwa dinamika pergerakan harga Bitcoin lebih banyak dipengaruhi oleh fluktuasi jangka pendek dan koreksi kesalahan sebelumnya, dibandingkan oleh pola harga historis yang stabil.

Nilai **AIC** dan **BIC** ditampilkan sebagai indikator kualitas model dan kompleksitasnya. Secara umum, nilai yang lebih rendah menunjukkan model yang lebih efisien dalam menjelaskan data. Namun, pada tahap ini nilai tersebut belum dibandingkan dengan konfigurasi ARIMA lainnya karena model ini hanya digunakan sebagai *baseline*.

Perlu dicatat bahwa parameter **(1,1,1)** belum tentu merupakan konfigurasi yang paling optimal. Untuk meningkatkan performa, biasanya dilakukan proses pencarian parameter secara sistematis, seperti *grid search* atau menggunakan **AutoARIMA**. Meski demikian, penggunaan ARIMA sederhana ini sudah cukup untuk memberikan gambaran awal tentang kemampuan model *time-series* dalam memprediksi harga Bitcoin.

Model ARIMA ini kemudian digunakan untuk menghasilkan prediksi pada data uji, sehingga performanya dapat dibandingkan dengan model lain seperti SARIMA dan Prophet pada tahap evaluasi berikutnya.

In [ ]:
# Forecasting ARIMA
arima_forecast = arima_fit.forecast(steps=len(test_data))
arima_rmse = np.sqrt(mean_squared_error(test_data['Price'], arima_forecast))
print(f"ARIMA RMSE: {arima_rmse:.2f}")

Pada tahap ini, model **ARIMA** digunakan untuk melakukan **prediksi harga Bitcoin** pada periode data uji (test set). Model menghasilkan prediksi untuk **24 bulan ke depan**, sesuai dengan panjang data uji yang telah ditentukan sebelumnya.

Setelah prediksi dihasilkan, performa model dievaluasi menggunakan metrik **RMSE (Root Mean Squared Error)**. RMSE mengukur seberapa besar rata-rata selisih antara harga hasil prediksi dengan harga aktual, dalam satuan yang sama dengan data aslinya (USD).

Hasil evaluasi menunjukkan nilai **RMSE sebesar 44.018**. Artinya, secara rata-rata, prediksi harga Bitcoin dari model ARIMA meleset sekitar **44 ribu USD** dari harga aktual pada periode pengujian.

Nilai RMSE yang cukup besar ini perlu dilihat dalam konteks karakteristik Bitcoin itu sendiri. Bitcoin memiliki harga yang sangat tinggi dan volatilitas yang ekstrem, sehingga kesalahan prediksi dalam puluhan ribu dolar masih tergolong wajar untuk model *time-series* sederhana seperti ARIMA.

Hasil ini menunjukkan bahwa ARIMA mampu menangkap **arah pergerakan umum** harga Bitcoin, namun masih memiliki keterbatasan dalam mengikuti fluktuasi harga yang tajam. Oleh karena itu, model ARIMA digunakan sebagai **baseline**, dan performanya akan dibandingkan dengan model lain yang lebih kompleks, seperti SARIMA dan Prophet, untuk melihat apakah pendekatan lain dapat memberikan hasil prediksi yang lebih akurat.


### 3.3 Model 2: SARIMA

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Menggunakan order (1,1,1) dan seasonal (1,1,1,12)
sarima_model = SARIMAX(train_data['Price'], 
                       order=(1, 1, 1), 
                       seasonal_order=(1, 1, 1, 12))
sarima_fit = sarima_model.fit(disp=False)

# Forecasting SARIMA
sarima_forecast = sarima_fit.forecast(steps=len(test_data))
sarima_rmse = np.sqrt(mean_squared_error(test_data['Price'], sarima_forecast))
print(f"SARIMA RMSE: {sarima_rmse:.2f}")

Pada tahap ini, digunakan model **SARIMA** sebagai pengembangan dari ARIMA dengan menambahkan unsur **musiman (seasonal)**. Model ini dirancang untuk menangkap pola yang berulang secara periodik, yang tidak dapat ditangani oleh ARIMA biasa.

Pada konfigurasi **SARIMA(1,1,1)(1,1,1,12)**:
- Komponen **(1,1,1)** merepresentasikan pola non-musiman, sama seperti pada model ARIMA
- Komponen **(1,1,1,12)** merepresentasikan pola musiman dengan siklus **12 bulan (tahunan)**

Penambahan angka **12** menunjukkan bahwa model mencoba mempelajari pola yang berulang setiap satu tahun. Hal ini relevan untuk data harga Bitcoin bulanan, karena pergerakan harga sering dipengaruhi oleh siklus tahunan, seperti sentimen pasar, kondisi makroekonomi, dan perilaku investor yang cenderung berulang dari tahun ke tahun.

Seperti pada ARIMA, proses **differencing** juga dilakukan secara otomatis di dalam model:
- **d = 1** menghilangkan tren jangka panjang
- **D = 1** menghilangkan pola musiman tahunan

Dengan demikian, SARIMA bekerja pada data yang telah distabilkan baik dari sisi tren maupun musiman.

Hasil evaluasi menunjukkan bahwa nilai **RMSE SARIMA sebesar 36.057**, lebih rendah dibandingkan RMSE ARIMA. Penurunan ini mengindikasikan bahwa dengan mempertimbangkan unsur musiman, model mampu menghasilkan prediksi yang lebih akurat.

Hal ini menunjukkan bahwa meskipun Bitcoin dikenal sangat volatil, terdapat pola musiman tahunan yang masih dapat ditangkap oleh model. Oleh karena itu, SARIMA menjadi alternatif yang lebih kuat dibandingkan ARIMA ketika data memiliki indikasi pola berulang dalam periode tertentu.

Model SARIMA ini selanjutnya dibandingkan dengan model Prophet untuk melihat apakah pendekatan yang lebih fleksibel terhadap tren dan perubahan pola dapat menghasilkan performa yang lebih baik.

### 3.4 Model 3: Prophet

In [ ]:
from prophet import Prophet

# Prophet expects columns ['ds', 'y']
prophet_train = train_data.reset_index().rename(columns={'date': 'ds', 'Price': 'y'})
prophet_test = test_data.reset_index().rename(columns={'date': 'ds', 'Price': 'y'})

# Bitcoin sering mengikuti tren non-linear, 'multiplicative' seasonality mungkin lebih cocok
m = Prophet(seasonality_mode='multiplicative')
m.fit(prophet_train)

future = m.make_future_dataframe(periods=len(test_data), freq='MS')
forecast = m.predict(future)

prophet_forecast = forecast.iloc[-len(test_data):]['yhat']
prophet_rmse = np.sqrt(mean_squared_error(test_data['Price'], prophet_forecast))
print(f"Prophet RMSE: {prophet_rmse:.2f}")

Pada tahap ini, digunakan model **Prophet**, yaitu model *time-series* yang dikembangkan oleh Facebook (Meta). Model ini dirancang untuk memudahkan proses peramalan data berbasis waktu, terutama pada data yang memiliki **tren non-linear**, **perubahan pola mendadak (changepoints)**, serta kemungkinan **data yang tidak selalu rapi**.

Berbeda dengan ARIMA dan SARIMA, Prophet tidak mengharuskan data bersifat stasioner dan tidak memerlukan proses *differencing* secara eksplisit. Model ini secara otomatis memisahkan data menjadi beberapa komponen utama, yaitu:
- **Tren jangka panjang**, yang bisa berubah seiring waktu
- **Pola musiman**, seperti pola tahunan atau bulanan
- **Efek kejadian khusus**, jika didefinisikan (misalnya hari besar atau peristiwa tertentu)

Pada implementasi ini, Prophet menggunakan **multiplicative seasonality**. Artinya, efek musiman dianggap berubah mengikuti level harga. Pendekatan ini cukup relevan untuk Bitcoin, karena fluktuasi harga cenderung semakin besar ketika harga berada pada level yang tinggi.

Setelah model dilatih menggunakan data latih, Prophet digunakan untuk memprediksi harga Bitcoin pada periode data uji. Hasil evaluasi menunjukkan nilai **RMSE sebesar 47.777**, yang lebih tinggi dibandingkan ARIMA dan SARIMA.

Hasil ini mengindikasikan bahwa meskipun Prophet sangat fleksibel dan kuat dalam menangani perubahan tren serta *changepoints*, pada kasus ini performanya kurang optimal untuk memprediksi harga Bitcoin bulanan. Salah satu kemungkinan penyebabnya adalah volatilitas Bitcoin yang ekstrem dan perubahan harga yang sangat tajam, sehingga sulit diikuti oleh pendekatan tren yang lebih halus seperti Prophet.

Dengan demikian, pada proyek ini Prophet berfungsi sebagai pembanding model modern, sementara model klasik seperti SARIMA justru menunjukkan performa yang lebih baik dalam menangkap pola harga Bitcoin bulanan.

## 4. Deployment & Evaluation

### 4.1 Model Evaluation

In [ ]:
results = pd.DataFrame({
    'Model': ['ARIMA', 'SARIMA', 'Prophet'],
    'RMSE': [arima_rmse, sarima_rmse, prophet_rmse]
})

results.sort_values(by='RMSE', ascending=True)

Pada tahap ini, dilakukan **perbandingan performa antar model** menggunakan metrik **RMSE (Root Mean Squared Error)**. Tujuan dari langkah ini adalah untuk menentukan model mana yang memberikan hasil prediksi paling akurat pada data uji.

Hasil perbandingan menunjukkan urutan performa sebagai berikut:
1. **SARIMA** – RMSE ≈ **36.071**
2. **ARIMA** – RMSE ≈ **44.035**
3. **Prophet** – RMSE ≈ **47.793**

Model dengan nilai RMSE paling kecil dianggap memiliki performa terbaik, karena rata-rata selisih antara nilai prediksi dan nilai aktual lebih rendah. Berdasarkan hasil ini, **SARIMA** menjadi model dengan performa paling baik dalam memprediksi harga Bitcoin bulanan pada periode pengujian.

Keunggulan SARIMA kemungkinan berasal dari kemampuannya menangkap **pola musiman tahunan (12 bulan)**, yang tidak dimodelkan secara eksplisit pada ARIMA dan ditangani secara lebih fleksibel namun kurang presisi oleh Prophet. Hal ini menunjukkan bahwa meskipun Bitcoin sangat volatil, masih terdapat pola musiman yang cukup konsisten pada data bulanan.

ARIMA, sebagai model *baseline*, memberikan performa yang cukup baik tetapi masih kalah dibandingkan SARIMA karena tidak mempertimbangkan unsur musiman. Sementara itu, Prophet—meskipun unggul dalam menangani tren non-linear dan *changepoints*—menunjukkan performa yang paling rendah pada kasus ini, kemungkinan karena volatilitas harga Bitcoin yang terlalu ekstrem untuk diikuti oleh pendekatan tren yang lebih halus.

Berdasarkan evaluasi ini, **SARIMA dipilih sebagai model terbaik** untuk digunakan pada tahap visualisasi akhir dan interpretasi bisnis, khususnya untuk melihat proyeksi tren harga Bitcoin dalam jangka menengah.


### 4.2 Final Forecast Visualization

In [ ]:
# Tentukan model terbaik secara otomatis
best_model_name = results.sort_values(by='RMSE').iloc[0]['Model']

if best_model_name == 'ARIMA':
    best_forecast = arima_forecast
elif best_model_name == 'SARIMA':
    best_forecast = sarima_forecast
else:
    best_forecast = prophet_forecast

# Plotting Comparison
fig = go.Figure()

# Actual Data
fig.add_trace(go.Scatter(x=test_data.index, y=test_data['Price'], 
                         mode='lines', name='Actual Price'))

# Best Model Forecast
fig.add_trace(go.Scatter(x=test_data.index, y=best_forecast, 
                         mode='lines', name=f'{best_model_name} Forecast', 
                         line=dict(color='red', dash='dot')))

fig.update_layout(title=f'Bitcoin Price Prediction: Actual vs {best_model_name}',
                  xaxis_title='Date',
                  yaxis_title='Price (USD)')
fig.show()

Pada tahap ini, dilakukan **visualisasi perbandingan antara harga Bitcoin aktual dan hasil prediksi dari model terbaik**, yaitu **SARIMA**. Model SARIMA dipilih secara otomatis berdasarkan nilai **RMSE terendah** dibandingkan model ARIMA dan Prophet.

Grafik ini menampilkan:
- **Garis biru (Actual Price)**: harga Bitcoin aktual pada periode data uji (Januari 2024 – Desember 2025)
- **Garis merah putus-putus (SARIMA Forecast)**: hasil prediksi harga Bitcoin dari model SARIMA pada periode yang sama

Tujuan utama visualisasi ini adalah untuk melihat **seberapa dekat prediksi model dengan pergerakan harga sebenarnya**, tidak hanya dari sisi angka evaluasi (RMSE), tetapi juga dari sisi pola dan arah tren.

Dari grafik terlihat bahwa:
- Model SARIMA cukup baik dalam menangkap **arah pergerakan harga secara umum** (naik atau turun)
- Namun, prediksi SARIMA cenderung **lebih halus** dibandingkan harga aktual
- Lonjakan harga yang sangat tajam pada data aktual tidak sepenuhnya diikuti oleh model

Hal ini wajar karena SARIMA bekerja dengan pola historis dan musiman, sehingga lebih fokus pada **tren rata-rata** dibandingkan perubahan ekstrem dalam waktu singkat. Dengan kata lain, model ini lebih cocok untuk:
- Melihat **arah tren jangka menengah**
- Memberikan gambaran pergerakan harga secara umum
- Mendukung analisis strategis, bukan prediksi harga harian yang presisi

Visualisasi ini memperkuat hasil evaluasi sebelumnya bahwa **SARIMA merupakan model terbaik dalam proyek ini**, meskipun tetap memiliki keterbatasan dalam menangkap volatilitas ekstrem Bitcoin. Oleh karena itu, hasil prediksi sebaiknya digunakan sebagai **indikator tren**, bukan sebagai angka harga pasti untuk pengambilan keputusan trading jangka pendek.


### 4.3 Save Artefak Joblib

In [ ]:
import os
import json
import joblib
from datetime import datetime

# =========================
# Model configuration
# =========================
MODEL_NAME = "sarima"
MODEL_DISPLAY_NAME = "SARIMA"
MODEL_ORDER = "(1,1,1)(1,1,1,12)"

# Create Models directory (one level above notebook)
MODEL_DIR = "..\\models"
os.makedirs(MODEL_DIR, exist_ok=True)

# Generate timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

MODEL_FILENAME = f"{MODEL_NAME}_model_{timestamp}.joblib"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)

joblib.dump(sarima_fit, MODEL_PATH)

print(f"{MODEL_DISPLAY_NAME} model successfully saved at: {MODEL_PATH}")

metadata = {
    "model_name": MODEL_DISPLAY_NAME,
    "model_key": MODEL_NAME,
    "model_order": MODEL_ORDER,
    "model_file": MODEL_FILENAME,
    "trained_at": timestamp,
    "train_period": {
        "start": str(train_data.index.min().date()),
        "end": str(train_data.index.max().date())
    },
    "test_period": {
        "start": str(test_data.index.min().date()),
        "end": str(test_data.index.max().date())
    },
    "evaluation_metric": "RMSE",
    "rmse": float(sarima_rmse),
    "data_frequency": "Monthly",
    "target_variable": "Bitcoin Close Price (USD)"
}

METADATA_FILENAME = f"{MODEL_NAME}_metadata_{timestamp}.json"
METADATA_PATH = os.path.join(MODEL_DIR, METADATA_FILENAME)

with open(METADATA_PATH, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Model metadata saved at: {METADATA_PATH}")

## 5. Stakeholder Acceptance

### 5.1 Conclusions

Proyek ini menunjukkan bahwa pendekatan *time-series forecasting* pada **data Bitcoin bulanan** mampu memberikan gambaran tren harga yang lebih jelas dibandingkan data harian yang sangat volatil. Dari tiga model yang diuji—**ARIMA, SARIMA, dan Prophet**—model **SARIMA** menunjukkan performa terbaik berdasarkan nilai **RMSE terendah**.

Keunggulan SARIMA terletak pada kemampuannya menangkap **pola musiman tahunan (12 bulan)** yang masih terlihat pada pergerakan harga Bitcoin. Hasil visualisasi juga menunjukkan bahwa SARIMA cukup baik dalam mengikuti **arah pergerakan harga secara umum**, sehingga cocok digunakan sebagai model dasar (*baseline*) untuk analisis tren jangka menengah.

### 5.2 Limitations

Meskipun memberikan hasil yang cukup baik, pendekatan dalam proyek ini memiliki beberapa keterbatasan:
- Model hanya memanfaatkan **data historis harga**, tanpa mempertimbangkan faktor eksternal seperti berita, regulasi, kondisi makroekonomi, atau sentimen pasar.
- Hasil prediksi cenderung **lebih halus** dan belum mampu menangkap lonjakan harga ekstrem yang sering terjadi pada Bitcoin.
- Analisis dilakukan pada skala **bulanan**, sehingga kurang sesuai untuk kebutuhan *trading* jangka pendek atau harian.
- Parameter model ARIMA dan SARIMA belum dioptimalkan secara menyeluruh melalui proses *grid search* atau AutoARIMA.

### 5.3 Recommendations

Berdasarkan hasil dan keterbatasan tersebut, beberapa rekomendasi yang dapat dipertimbangkan adalah:
- Menggunakan hasil prediksi sebagai **indikator arah tren**, bukan sebagai angka harga pasti.
- Mengombinasikan model *time-series* dengan **variabel eksternal** seperti volume, indikator makroekonomi, atau sentimen pasar.
- Melakukan **optimasi parameter model** untuk meningkatkan akurasi prediksi.
- Mengembangkan pendekatan **hybrid**, misalnya menggabungkan SARIMA dengan model *machine learning*.
- Menyesuaikan horizon prediksi dengan kebutuhan bisnis, khususnya untuk analisis strategis jangka menengah hingga panjang.
